# Pruebas de Escritorio y Validación del API - Endpoint /verificar-persona

Este notebook contiene pruebas interactivas para validar el funcionamiento del endpoint `/verificar-persona` de la API de OCR y validación de documentos.

## Objetivos:
- Validar la lógica de puntuación de documentos y nombres
- Probar diferentes escenarios de matching
- Verificar el comportamiento con parámetros opcionales
- Realizar pruebas de integración con el API

## 1. Configuración del Entorno

In [12]:
# Instalar dependencias si es necesario
# !pip install -r requirements.txt

# Importar librerías necesarias
import requests
import json
import time
from IPython.display import JSON, display, HTML

# Configuración del servidor local
BASE_URL = "http://localhost:8000"  # Cambia si usas otro puerto
ENDPOINT = "/verificar-persona"

print("Configuración completada")
print(f"API Base URL: {BASE_URL}")
print(f"Endpoint: {ENDPOINT}")

Configuración completada
API Base URL: http://localhost:8000
Endpoint: /verificar-persona


## 2. Funciones de Utilidad

In [13]:
def verificar_persona_api(nombre, documento, texto_evaluar, usar_bono=True, umbral_fuzzy=0.88):
    """
    Función para hacer requests al endpoint /verificar-persona
    """
    url = f"{BASE_URL}{ENDPOINT}"
    params = {
        "usar_bono": str(usar_bono).lower(),
        "umbral_fuzzy": umbral_fuzzy
    }
    
    payload = {
        "nombre": nombre,
        "documento": documento,
        "texto_evaluar": texto_evaluar
    }
    
    try:
        response = requests.post(url, json=payload, params=params)
        return {
            "status_code": response.status_code,
            "success": response.status_code == 200,
            "data": response.json() if response.status_code == 200 else None,
            "error": response.text if response.status_code != 200 else None
        }
    except Exception as e:
        return {
            "status_code": None,
            "success": False,
            "data": None,
            "error": str(e)
        }

def mostrar_resultado(resultado, titulo="Resultado", mostrar_json=True):
    """
    Función para mostrar resultados de manera formateada
    """
    print(f"\n{'='*50}")
    print(f"{titulo}")
    print(f"{'='*50}")
    
    if resultado["success"]:
        print(f"✅ Status: {resultado['status_code']}")
        data = resultado["data"]
        print(f"📊 Score Total: {data['score']}/100")
        print(f"\n📋 Componentes:")
        print(f"   📄 Documento: {data['componentes']['documento']} pts")
        print(f"   👤 Nombre: {data['componentes']['nombre']} pts")
        print(f"   🎯 Bono proximidad: {data['componentes']['bono_proximidad']} pts")
        
        if data['doc_match']['numero_encontrado']:
            print(f"\n🔍 Documento encontrado: {data['doc_match']['numero_encontrado']}")
        
        if data['tokens_encontrados']:
            print(f"🔍 Tokens de nombre encontrados: {', '.join(data['tokens_encontrados'])}")
            
        # Mostrar detalles del nombre
        if 'nombre_match' in data and 'detalles' in data['nombre_match']:
            print(f"\n📝 Detalles del matching de nombre:")
            for detalle in data['nombre_match']['detalles']:
                print(f"   - '{detalle['token']}': {detalle['tipo']}")
        
        # Mostrar JSON completo si se solicita
        if mostrar_json:
            print(f"\n📄 JSON Response completo:")
            print(json.dumps(data, indent=2, ensure_ascii=False))
    else:
        print(f"❌ Error: {resultado['error']}")
        print(f"Status Code: {resultado['status_code']}")
        if mostrar_json and resultado.get("error"):
            print(f"\n📄 Error Response:")
            print(resultado["error"])

def probar_caso(nombre, documento, texto, titulo, usar_bono=True, umbral_fuzzy=0.88, mostrar_json=True):
    """
    Función para probar un caso específico
    """
    resultado = verificar_persona_api(nombre, documento, texto, usar_bono, umbral_fuzzy)
    mostrar_resultado(resultado, titulo, mostrar_json)
    return resultado

## 3. Pruebas de Escritorio - Lógica de Puntuación

### Reglas de Puntuación:
- **Documento**: 60 pts (completo) / 45 pts (primeros 6 dígitos) / 0 pts
- **Nombre**: 40 pts distribuidos entre tokens útiles
- **Bono proximidad**: +5 pts máximo si documento encontrado y ≥50% tokens nombre
- **Total**: Máximo 100 pts

In [14]:
# Prueba 1: Matching completo (esperado: 100 pts)
resultado1 = probar_caso(
    nombre="juan perez",
    documento="123456789",
    texto="juan perez documento 123456789",
    titulo="Prueba 1: Matching Completo"
)


Prueba 1: Matching Completo
✅ Status: 200
📊 Score Total: 100/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 40 pts
   🎯 Bono proximidad: 5 pts

🔍 Documento encontrado: 123456789
🔍 Tokens de nombre encontrados: juan, perez

📝 Detalles del matching de nombre:
   - 'juan': exacto
   - 'perez': exacto

📄 JSON Response completo:
{
  "score": 100,
  "componentes": {
    "documento": 60,
    "nombre": 40,
    "bono_proximidad": 5
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "123456789",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 40,
    "tokens_encontrados": [
      "juan",
      "perez"
    ],
    "tokens_fallidos": [],
    "detalles": [
      {
        "token": "juan",
        "match": "juan",
        "tipo": "exacto"
      },
      {
        "token": "perez",
        "match": "perez",
        "tipo": "exacto"
      }
    ]
  },
  "tokens_encontrados": [
    "juan",
    "perez"
  ],
  "documento_encontrado": "1

In [15]:
# Prueba 2: Matching parcial de documento (esperado: 85 pts)
resultado2 = probar_caso(
    nombre="juan",
    documento="123456789",
    texto="juan documento 123456",
    titulo="Prueba 2: Documento Parcial + Nombre"
)


Prueba 2: Documento Parcial + Nombre
✅ Status: 200
📊 Score Total: 90/100

📋 Componentes:
   📄 Documento: 45 pts
   👤 Nombre: 40 pts
   🎯 Bono proximidad: 5 pts

🔍 Documento encontrado: 123456
🔍 Tokens de nombre encontrados: juan

📝 Detalles del matching de nombre:
   - 'juan': exacto

📄 JSON Response completo:
{
  "score": 90,
  "componentes": {
    "documento": 45,
    "nombre": 40,
    "bono_proximidad": 5
  },
  "doc_match": {
    "puntos": 45,
    "coincidencia": "first6",
    "numero_encontrado": "123456",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 40,
    "tokens_encontrados": [
      "juan"
    ],
    "tokens_fallidos": [],
    "detalles": [
      {
        "token": "juan",
        "match": "juan",
        "tipo": "exacto"
      }
    ]
  },
  "tokens_encontrados": [
    "juan"
  ],
  "documento_encontrado": "123456"
}


In [16]:
# Prueba 3: Sin matching (esperado: 0 pts)
resultado3 = probar_caso(
    nombre="pedro",
    documento="999999999",
    texto="juan documento 123456",
    titulo="Prueba 3: Sin Matching"
)


Prueba 3: Sin Matching
✅ Status: 200
📊 Score Total: 0/100

📋 Componentes:
   📄 Documento: 0 pts
   👤 Nombre: 0 pts
   🎯 Bono proximidad: 0 pts

📝 Detalles del matching de nombre:
   - 'pedro': no_encontrado

📄 JSON Response completo:
{
  "score": 0,
  "componentes": {
    "documento": 0,
    "nombre": 0,
    "bono_proximidad": 0
  },
  "doc_match": {
    "puntos": 0,
    "coincidencia": null,
    "numero_encontrado": null,
    "pos": []
  },
  "nombre_match": {
    "puntos": 0,
    "tokens_encontrados": [],
    "tokens_fallidos": [
      "pedro"
    ],
    "detalles": [
      {
        "token": "pedro",
        "match": null,
        "tipo": "no_encontrado"
      }
    ]
  },
  "tokens_encontrados": [],
  "documento_encontrado": null
}


In [17]:
# Prueba 4: Matching difuso de nombre (esperado: ≥60 pts)
resultado4 = probar_caso(
    nombre="juan",
    documento="123456789",
    texto="juanito documento 123456789",
    titulo="Prueba 4: Matching Difuso de Nombre"
)


Prueba 4: Matching Difuso de Nombre
✅ Status: 200
📊 Score Total: 60/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 0 pts
   🎯 Bono proximidad: 0 pts

🔍 Documento encontrado: 123456789

📝 Detalles del matching de nombre:
   - 'juan': no_encontrado

📄 JSON Response completo:
{
  "score": 60,
  "componentes": {
    "documento": 60,
    "nombre": 0,
    "bono_proximidad": 0
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "123456789",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 0,
    "tokens_encontrados": [],
    "tokens_fallidos": [
      "juan"
    ],
    "detalles": [
      {
        "token": "juan",
        "match": null,
        "tipo": "no_encontrado"
      }
    ]
  },
  "tokens_encontrados": [],
  "documento_encontrado": "123456789"
}


## 4. Pruebas con Parámetros Opcionales

In [18]:
# Prueba 5: Sin bono de proximidad
resultado5 = probar_caso(
    nombre="juan perez",
    documento="123456789",
    texto="juan perez documento 123456789",
    titulo="Prueba 5: Sin Bono de Proximidad",
    usar_bono=False
)


Prueba 5: Sin Bono de Proximidad
✅ Status: 200
📊 Score Total: 100/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 40 pts
   🎯 Bono proximidad: 0 pts

🔍 Documento encontrado: 123456789
🔍 Tokens de nombre encontrados: juan, perez

📝 Detalles del matching de nombre:
   - 'juan': exacto
   - 'perez': exacto

📄 JSON Response completo:
{
  "score": 100,
  "componentes": {
    "documento": 60,
    "nombre": 40,
    "bono_proximidad": 0
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "123456789",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 40,
    "tokens_encontrados": [
      "juan",
      "perez"
    ],
    "tokens_fallidos": [],
    "detalles": [
      {
        "token": "juan",
        "match": "juan",
        "tipo": "exacto"
      },
      {
        "token": "perez",
        "match": "perez",
        "tipo": "exacto"
      }
    ]
  },
  "tokens_encontrados": [
    "juan",
    "perez"
  ],
  "documento_encontrado

In [19]:
# Prueba 6: Umbral fuzzy más estricto
resultado6 = probar_caso(
    nombre="juan",
    documento="123456789",
    texto="juanito documento 123456789",
    titulo="Prueba 6: Umbral Fuzzy Estricto (0.95)",
    umbral_fuzzy=0.95
)


Prueba 6: Umbral Fuzzy Estricto (0.95)
✅ Status: 200
📊 Score Total: 60/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 0 pts
   🎯 Bono proximidad: 0 pts

🔍 Documento encontrado: 123456789

📝 Detalles del matching de nombre:
   - 'juan': no_encontrado

📄 JSON Response completo:
{
  "score": 60,
  "componentes": {
    "documento": 60,
    "nombre": 0,
    "bono_proximidad": 0
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "123456789",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 0,
    "tokens_encontrados": [],
    "tokens_fallidos": [
      "juan"
    ],
    "detalles": [
      {
        "token": "juan",
        "match": null,
        "tipo": "no_encontrado"
      }
    ]
  },
  "tokens_encontrados": [],
  "documento_encontrado": "123456789"
}


## 5. Pruebas de Edge Cases

In [20]:
# Prueba 7: Datos vacíos
resultado7 = probar_caso(
    nombre="",
    documento="",
    texto="",
    titulo="Prueba 7: Datos Vacíos"
)


Prueba 7: Datos Vacíos
✅ Status: 200
📊 Score Total: 0/100

📋 Componentes:
   📄 Documento: 0 pts
   👤 Nombre: 0 pts
   🎯 Bono proximidad: 0 pts

📝 Detalles del matching de nombre:

📄 JSON Response completo:
{
  "score": 0,
  "componentes": {
    "documento": 0,
    "nombre": 0,
    "bono_proximidad": 0
  },
  "doc_match": {
    "puntos": 0,
    "coincidencia": null,
    "numero_encontrado": null,
    "pos": []
  },
  "nombre_match": {
    "puntos": 0,
    "tokens_encontrados": [],
    "tokens_fallidos": [],
    "detalles": []
  },
  "tokens_encontrados": [],
  "documento_encontrado": null
}


In [21]:
# Prueba 8: Texto con ruido
resultado8 = probar_caso(
    nombre="maria garcia",
    documento="987654321",
    texto="en el documento de maria garcia numero 987654321 se encuentra la informacion solicitada",
    titulo="Prueba 8: Texto con Ruido"
)


Prueba 8: Texto con Ruido
✅ Status: 200
📊 Score Total: 100/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 40 pts
   🎯 Bono proximidad: 5 pts

🔍 Documento encontrado: 987654321
🔍 Tokens de nombre encontrados: maria, garcia

📝 Detalles del matching de nombre:
   - 'maria': exacto
   - 'garcia': exacto

📄 JSON Response completo:
{
  "score": 100,
  "componentes": {
    "documento": 60,
    "nombre": 40,
    "bono_proximidad": 5
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "987654321",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 40,
    "tokens_encontrados": [
      "maria",
      "garcia"
    ],
    "tokens_fallidos": [],
    "detalles": [
      {
        "token": "maria",
        "match": "maria",
        "tipo": "exacto"
      },
      {
        "token": "garcia",
        "match": "garcia",
        "tipo": "exacto"
      }
    ]
  },
  "tokens_encontrados": [
    "maria",
    "garcia"
  ],
  "documento_encon

In [22]:
# Prueba 9: Nombre con stopwords
resultado9 = probar_caso(
    nombre="juan de la cruz",
    documento="111222333",
    texto="juan cruz documento 111222333",
    titulo="Prueba 9: Nombre con Stopwords"
)


Prueba 9: Nombre con Stopwords
✅ Status: 200
📊 Score Total: 100/100

📋 Componentes:
   📄 Documento: 60 pts
   👤 Nombre: 40 pts
   🎯 Bono proximidad: 5 pts

🔍 Documento encontrado: 111222333
🔍 Tokens de nombre encontrados: juan, cruz

📝 Detalles del matching de nombre:
   - 'juan': exacto
   - 'cruz': exacto

📄 JSON Response completo:
{
  "score": 100,
  "componentes": {
    "documento": 60,
    "nombre": 40,
    "bono_proximidad": 5
  },
  "doc_match": {
    "puntos": 60,
    "coincidencia": "full",
    "numero_encontrado": "111222333",
    "pos": [
      0
    ]
  },
  "nombre_match": {
    "puntos": 40,
    "tokens_encontrados": [
      "juan",
      "cruz"
    ],
    "tokens_fallidos": [],
    "detalles": [
      {
        "token": "juan",
        "match": "juan",
        "tipo": "exacto"
      },
      {
        "token": "cruz",
        "match": "cruz",
        "tipo": "exacto"
      }
    ]
  },
  "tokens_encontrados": [
    "juan",
    "cruz"
  ],
  "documento_encontrado": "1112

## 6. Pruebas de Rendimiento

In [23]:
# Prueba de rendimiento con múltiples requests
def prueba_rendimiento(num_requests=10):
    print(f"\n🚀 Ejecutando {num_requests} requests para medir rendimiento...")
    
    tiempos = []
    for i in range(num_requests):
        start_time = time.time()
        resultado = verificar_persona_api(
            "juan perez", "123456789", "juan perez documento 123456789"
        )
        end_time = time.time()
        tiempos.append(end_time - start_time)
        
        if not resultado["success"]:
            print(f"❌ Request {i+1} falló")
            break
    
    if tiempos:
        avg_time = sum(tiempos) / len(tiempos)
        min_time = min(tiempos)
        max_time = max(tiempos)
        
        print(f"\n📊 Estadísticas de Rendimiento:")
        print(f"   Promedio: {avg_time:.4f} segundos")
        print(f"   Mínimo: {min_time:.4f} segundos")
        print(f"   Máximo: {max_time:.4f} segundos")
        print(f"   Total requests exitosos: {len(tiempos)}/{num_requests}")

# Ejecutar prueba de rendimiento
prueba_rendimiento(5)


🚀 Ejecutando 5 requests para medir rendimiento...

📊 Estadísticas de Rendimiento:
   Promedio: 2.0409 segundos
   Mínimo: 2.0302 segundos
   Máximo: 2.0524 segundos
   Total requests exitosos: 5/5


## 7. Validación del API - Checklist

In [24]:
# Función para validar que el API esté funcionando correctamente
def validar_api():
    print("🔍 Validando funcionamiento del API...\n")
    
    checks = [
        {
            "name": "API reachable",
            "test": lambda: requests.get(f"{BASE_URL}/").status_code == 200
        },
        {
            "name": "Endpoint /verificar-persona exists",
            "test": lambda: verificar_persona_api("test", "test", "test")["status_code"] is not None
        },
        {
            "name": "Full match returns 100",
            "test": lambda: verificar_persona_api("juan", "123", "juan 123")["data"]["score"] == 100
        },
        {
            "name": "No match returns 0",
            "test": lambda: verificar_persona_api("xxx", "999", "yyy 111")["data"]["score"] == 0
        },
        {
            "name": "Response structure correct",
            "test": lambda: all(key in verificar_persona_api("a", "b", "c")["data"] for key in ["score", "componentes", "doc_match", "nombre_match"])
        }
    ]
    
    passed = 0
    total = len(checks)
    
    for check in checks:
        try:
            result = check["test"]()
            if result:
                print(f"✅ {check['name']}")
                passed += 1
            else:
                print(f"❌ {check['name']}")
        except Exception as e:
            print(f"❌ {check['name']}: Error - {str(e)}")
    
    print(f"\n📊 Resultado: {passed}/{total} checks pasaron")
    
    if passed == total:
        print("🎉 ¡API validado exitosamente!")
    else:
        print("⚠️  Algunos checks fallaron. Revisa la configuración.")

# Ejecutar validación
validar_api()

🔍 Validando funcionamiento del API...

✅ API reachable
✅ Endpoint /verificar-persona exists
✅ Full match returns 100
✅ No match returns 0
✅ Response structure correct

📊 Resultado: 5/5 checks pasaron
🎉 ¡API validado exitosamente!


## 8. Instrucciones de Uso

### Para usar este notebook:

1. **Iniciar el servidor API**:
   ```bash
   cd /ruta/a/pdf2Image
   uvicorn app.main:app --reload
   ```

2. **Ejecutar las celdas** en orden desde la celda 1

3. **Verificar resultados** en las salidas de cada celda

4. **Personalizar pruebas** modificando los parámetros en las funciones de prueba

### Casos de prueba incluidos:
- ✅ Matching completo (100 pts)
- ✅ Matching parcial (85 pts)
- ✅ Sin matching (0 pts)
- ✅ Matching difuso
- ✅ Parámetros opcionales
- ✅ Edge cases
- ✅ Rendimiento
- ✅ Validación del API

### Notas importantes:
- Asegúrate de que el servidor esté corriendo en `http://localhost:8000`
- Los textos deben estar **previamente limpiados** (sin tildes, minúsculas)
- El endpoint espera JSON con campos: `nombre`, `documento`, `texto_evaluar`
- Parámetros opcionales: `usar_bono` (boolean), `umbral_fuzzy` (float)
- **JSON Response**: Cada prueba muestra el JSON completo de respuesta para inspección detallada